<a href="https://colab.research.google.com/github/Victor-LE-GALL/Projet-Data-Analyst/blob/main/Data_Analyst_Tetes_de_series_Tous_tournois_Tennis_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install html2text

In [ ]:
# on importe toutes les librairies nécessaires

import requests
import bs4 as bs
import json
import html2text
from urllib.parse import parse_qs
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime 

In [ ]:
def tete_serie(annee, tournoi, url):      # fonction qui renvoie un dataframe comprenant les matchs ATP à l'aide d'un lien passé en paramètre

  #requête à partir du lien passé en paramètre. Le lien correspond à un tournoi pour une année
  headers = {'X-Fsign': 'SW9D1eZo'}
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.content, 'html.parser')
  soup=str(soup)

  #création d'un dataframe vide
  d = {'Année': [], 'Tournoi': [], 'Tour':[], 'Joueur 1':[], 'Joueur 2':[], 'Res 1':[], 'Res 2':[], 'Cote 1':[], 'Cote 2':[]}
  df_matchs = pd.DataFrame(data=d)

  # à partir du contenu du résultat de la requête :

  soup_joueurs=soup.split('¬')[0]
  joueurs=soup_joueurs.split('_')

  tours=soup.split('RN÷')[1].split('¬')[0]
  tours=tours.split('|')

  soup_matchs=soup.split('RI÷')
  for i in range(0, len(soup_matchs)):
    for j in range(0, len(tours)):
      if soup_matchs[i].split('¬')[0]==tours[j].split('_')[0]:
        round=tours[j].split('_')[1]
        if '64' in round:
          round='64'
        if '32' in round:
          round='32'
        if '16' in round:
          round='16'
        if '8' in round:
          round='8'
        if 'Quarts' in round:
          round='4'
        if 'Demi' in round:
          round='2'
        if round=='Finale':
          round='1'
        
        match=soup_matchs[i].split('HP÷')
        for k in range(1, len(match)):
          
          if 'RE' in match[k] and 'RT' in match[k]:
            tete_serie_1=match[k].split('HI÷(')
            tete_serie_2=match[k].split('AI÷(')

            # on récupère l'identifiant du match
            code_match=match[k].split('AA÷')[1].split('¬')[0]

            t1=0
            t2=0

            if len(tete_serie_1)>1:
              t1=tete_serie_1[1].split(')¬')[0]
            
            if len(tete_serie_2)>1:
              t2=tete_serie_2[1].split(')¬')[0]

            
            r1=match[k].split('RE÷')[1].split('¬')[0]
            r2=match[k].split('RT÷')[1].split('¬')[0]




            # requête avec l'identifiant du match (pour obtenir les cotes)
            url_cote='https://d.flashscore.fr/x/feed/df_od_1_' + code_match
            r_c = requests.get(url_cote, headers=headers) 
            soup_c = BeautifulSoup(r_c.content, 'html.parser')
            sc=str(soup_c)
            scb=sc.split('1. set')
            scb=scb[0]
            cote_1tot=0
            cote_2tot=0
            cote_1_max=0
            cote_2_max=0

            #total des cotes proposées par les bookmakers pour le joueur 1
            scba=scb.split('XB÷')
            for sci in range(1, len(scba)):
              scca=scba[sci].split('¬')
              scca1=scca[0].split(']')
              if len(scca1)>1:
                if float(scca1[1])>cote_1_max:
                  cote_1_max=float(scca1[1])
                cote_1tot=cote_1tot+float(scca1[1])
              else:
                if float(scca[0])>cote_1_max:
                  cote_1_max=float(scca[0])
                cote_1tot=cote_1tot+float(scca[0])
            
            #total des cotes proposées par les bookmakers pour le joueur 1
            scbb=scb.split('XC÷')
            for sci in range(1, len(scbb)):
              sccb=scbb[sci].split('¬')
              sccb1=sccb[0].split(']')
              if len(sccb1)>1:
                if float(sccb1[1])>cote_2_max:
                  cote_2_max=float(sccb1[1])
                cote_2tot=cote_2tot+float(sccb1[1])
              else:
                if float(sccb[0])>cote_2_max:
                  cote_2_max=float(sccb[0])
                cote_2tot=cote_2tot+float(sccb[0])

            # calcul de la cote moyenne pour chacun des joueurs
            if len(scba)>1 and len(scbb)>1:
              cote_1=cote_1tot/(len(scba)-1)
              cote_2=cote_2tot/(len(scbb)-1)

            # on crée une nouvelle ligne au dataframe comprenant les informations du match
            print(annee, tournoi, round, t1, t2, r1, r2, cote_1, cote_2)
            df_matchs=df_matchs.append({'Année': annee, 'Tournoi': tournoi, 'Tour':round, 'Joueur 1':t1, 'Joueur 2':t2, 'Res 1':r1, 'Res 2':r2, 'Cote 1':cote_1, 'Cote 2':cote_2}, ignore_index=True)


  return df_matchs

In [ ]:
url='https://www.flashscore.fr/x/req/m_2_5724'    # url de départ (page d'accueil des résultats de Tennis)

df_tot=[] # liste qui comprendra tous les matchs
co=0

headers = {'X-Fsign': 'SW9D1eZo'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
soup=str(soup)


sp=soup.split('MU÷')

for i in range(1, len(sp)):  # itération sur tous les tournois
  nom_tournoi=sp[i].split('¬')[0]

  for annee in range(2022, 2011, -1): # itération sur les années pour le tournoi (de 2022 à 2012)
    try :
      url_tournoi='https://www.flashscore.fr/tennis/atp-simples/'+nom_tournoi+'-'+str(annee)+'/'
      headers = {'X-Fsign': 'SW9D1eZo'}
      r_tou = requests.get(url_tournoi, headers=headers)
      soup_tou = BeautifulSoup(r_tou.content, 'html.parser')
      soup_tou=str(soup_tou)
      st1=soup_tou.split('"tournament\":\"')[1].split('\"')[0]
      st2=soup_tou.split('tournamentStage\":\"')[1].split('\"')[0]
      code1=st1
      code2=st2

      url_archive='https://d.flashscore.fr/x/feed/dr_'+ code1+'_'+code2

      df_tournoi=tete_serie(int(annee), nom_tournoi ,url_archive) # on crée un dataframe avec les résultats de ce tournoi pour cette année

      df_tot.append(df_tournoi) # on ajoute ces matchs à une liste
    except:
      pass
      

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2013 stuttgart 8 6 0 0 2 2.3875 1.5275
2013 stuttgart 8 0 2 0 2 8.6875 1.06
2013 stuttgart 4 1 5 0 2 1.33 3.2
2013 stuttgart 4 RET. 0 0 1 2.975 1.3625
2013 stuttgart 4 0 4 2 0 2.425 1.5275
2013 stuttgart 4 0 2 0 2 1.4425 2.6375
2013 stuttgart 2 5 0 2 0 1.3625 3.0375
2013 stuttgart 2 0 2 0 2 3.275 1.3150000000000002
2013 stuttgart 1 5 2 2 1 2.045 1.725
https://d.flashscore.fr/x/feed/dr_ALjsz23i_zs3asL4S
2012 stuttgart 16 0 0 2 0 2.135 1.6575
2012 stuttgart 16 0 Q 2 0 1.4449999999999998 2.6475
2012 stuttgart 16 0 5 2 0 2.06 1.6975
2012 stuttgart 16 0 WC 2 0 1.0275 12.875
2012 stuttgart 16 Q 0 0 2 2.2725 1.5850000000000002
2012 stuttgart 16 0 7 2 1 1.9049999999999998 1.8075
2012 stuttgart 16 6 WC 1 2 1.3500000000000003 3.05
2012 stuttgart 16 0 LL 2 0 1.3500000000000003 3.05
2012 stuttgart 16 0 0 2 0 1.3750000000000002 2.9325
2012 stuttgart 16 8 Q 2 1 1.3875000000000002 2.8449999999999998
2012 stuttgart 16 0 Q 1 

In [ ]:
# création du grand dataframe qui comprendra tous les matchs depuis 2012 (on récupére les données précédemment ajouté à la liste "df_tot")
d = {'Année': [], 'Tournoi': [], 'Tour':[], 'Joueur_1':[], 'Joueur_2':[], 'Res_1':[], 'Res_2':[], 'Cote_1':[], 'Cote_2':[]}
df= pd.DataFrame(data=d)
for i in range(0, len(df_tot)):
  df_ind=df_tot[i]
  for j in range(0, len(df_ind)):
    annee=df_ind.at[j, 'Année']
    tournoi=df_ind.at[j, 'Tournoi']
    round=df_ind.at[j, 'Tour']
    t1=df_ind.at[j, 'Joueur 1']
    t2=df_ind.at[j, 'Joueur 2']
    r1=df_ind.at[j, 'Res 1']
    r2=df_ind.at[j, 'Res 2']
    cote_1=df_ind.at[j, 'Cote 1']
    cote_2=df_ind.at[j, 'Cote 2']
    df=df.append({'Année': annee, 'Tournoi': tournoi, 'Tour':round, 'Joueur_1':t1, 'Joueur_2':t2, 'Res_1':r1, 'Res_2':r2, 'Cote_1':cote_1, 'Cote_2':cote_2}, ignore_index=True)

,Année,Tournoi,Tour,Joueur_1,Joueur_2,Res_1,Res_2,Cote_1,Cote_2
0,2021.0,acapulco,16,1,0.0,2,0,1.0540,10.650000
1,2021.0,acapulco,16,0,0.0,2,0,1.5480,2.468000
2,2021.0,acapulco,16,0,WC,0,2,2.0580,1.766000
3,2021.0,acapulco,16,0,7,0,2,3.9980,1.252000
4,2021.0,acapulco,16,3,Q,1,2,1.2680,3.828000
...,...,...,...,...,...,...,...,...,...
24831,2019.0,zhuhai,4,7,4,2,1,1.6825,2.162500
24832,2019.0,zhuhai,4,0,2,0,2,4.2720,1.228000
24833,2019.0,zhuhai,2,0,8,2,1,1.9500,1.860000
24834,2019.0,zhuhai,2,7,2,2,0,2.3100,1.633333


In [ ]:
# on affiche les tournois
print('Liste des tournois :')
liste_tournoi=df['Tournoi'].unique()
for i in liste_tournoi:
  print(i)


# on retire les tournois spéciaux
df.drop(df[df.Tournoi == 'finals-turin'].index, inplace=True)
df.drop(df[df.Tournoi == 'jeux-olympiques'].index, inplace=True)
df.drop(df[df.Tournoi == 'next-gen-finals-milan'].index, inplace=True)
df=df.reset_index()


# on traite les cas particuliers pour les têtes de serie
for i in range(0, len(df)):
  if 'RET' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']='RET'
  if 'RET' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']='RET'
  if '0.0' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=0
  if '0.0' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=0
  if 'Q' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=34
  if 'Q' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=34
  if 'WC' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=35
  if 'WC' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=35
  if 'LL' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=36
  if 'LL' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=36
  if 'PR' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=37
  if 'PR' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=37
  if 'SE' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=38
  if 'SE' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=38
  if 'Alt' in str(df.at[i, 'Joueur_1']):
    df.at[i, 'Joueur_1']=39
  if 'Alt' in str(df.at[i, 'Joueur_2']):
    df.at[i, 'Joueur_2']=39


# on supprime les matchs non terminés
df.drop(df[df.Joueur_1 == 'RET'].index, inplace=True)
df.drop(df[df.Joueur_2 == 'RET'].index, inplace=True)
df=df.reset_index()





print(df['Res_1'].min(), df['Res_1'].max())
print(df['Res_2'].min(), df['Res_2'].max())
print(df['Cote_1'].min(), df['Cote_1'].max())
print(df['Cote_2'].min(), df['Cote_2'].max())

# on arrondit les cotes à 2 chiffres après la virgule
df['Cote_1']=df['Cote_1'].round(2)
df['Cote_2']=df['Cote_2'].round(2)

# on impose les types de données corrects pour les colonnes pour lesquelles c'est nécessaire
df['Joueur_1']=df['Joueur_1'].astype('int')
df['Joueur_2']=df['Joueur_2'].astype('int')
df['Année']=df['Année'].astype('int')
df['Tour']=df['Tour'].astype('int')
df['Tournoi']=df['Tournoi'].astype('string')
print('types de données par colonnes', df.dtypes)

df=df.drop(['level_0', 'index'], axis=1)

# on ajoute une colonne pour le type de tournoi
df['Tournoi_type']=0
liste_grand_chelem=['open-d-australie', 'internationaux-de-france', 'wimbledon', 'us-open']
liste_masters_1000=['cincinnati', 'indian-wells', 'madrid', 'masters-de-paris-bercy', 'miami', 'monte-carlo', 'montreal', 'shanghai', 'rome', 'toronto']
for i in range(0, len(df)):
  if df.at[i, 'Tournoi'] in liste_grand_chelem:
    df.at[i, 'Tournoi_type']=2
  if df.at[i, 'Tournoi'] in liste_masters_1000:
    df.at[i, 'Tournoi_type']=1
    
print('nombre de matchs par type de tournoi:', df['Tournoi_type'].value_counts())

In [ ]:
# enregistrement du dataframe en csv

path_drive_tennis='/content/drive/MyDrive/Tennis/'
df.to_csv(path_drive_tennis+'tetes_de_serie_tout_v2.csv', index=False)